## Data Engineer Interview - Quiz 2

### E-Commerce Returns & Partial Refunds - You’re tasked with modeling e-commerce orders, including the ability to handle multiple shipments and partial returns per order. **Question: **Describe the schema for Orders, Shipments, Returns, and Payments. How would you avoid data inconsistencies with partial refunds over time?

Tables to use for modeling:

1. fact_orders - order activity by customer

- order_id (PK)
- order_number
- customer_id (FK)
- order_date
- order_status (completed, partially_shipped, closed)
- order_currency
- order_gross_amount
- order_tax_amount
- order_net_amount
- created_at

Rule: Order totals are immutable after checkout; adjustments happen via returns/refunds.

2. fact_order_items - line-level granularity is required for partial shipments and returns.

- order_item_id (PK)
- order_items
- order_id (FK)
- sku
- unit_price
- quantity_ordered

3. fact_shipments - tracks fulfillment of order events; Relationship: one order can have many shipments.

- shipment_id (PK)
- order_id (FK)
- shipment_status (created, shipped, delivered)
- shipped_at
- tracking_number
- quantity_shipped

4. fact_returns - tracks return events.

- return_id (PK)
- order_id (FK)
- return_status (requested, approved, received, closed)
- return_reason
- created_at
- quantity_returned

Rule: Allows multiple partial returns for the same item over time.

5. fact_payments_refunds - use a ledger model instead of mutable balances.

- payment_id (PK)
- order_id (FK)
- payment_method
- payment_amount
- payment_status (completed, rejected, pending)
- paid_at
- refund_id (PK)
- return_id (FK)
- refund_amount
- refund_status (approved, completed, rejected)
- refunded_at

Avoiding Inconsistencies:

- Keep orders and payments immutable; refunds are separate records.
- Track returns at item level to allow partial returns.

Enforce Quantity Constraints:

- total qty_returned ≤ qty_ordered
- total items_shipped ≥ total items_returned

Derive current balance as:

- sum(payments) − sum(refunds) ✅